First we will load and decompress all of our data.

In [ ]:
!curl https://ecommwebsitedata.s3.amazonaws.com/acm-sf-chapter-hackathon-big.zip --output bestbuy.zip
!ls
!unzip bestbuy.zip
!tar xvf product_data.tar.gz

Next, we will import any modules that we may need and define a variable to hold the path to our data file of interest.

In [12]:
import pandas as pd
import xml.etree.ElementTree as ETree
CAT_FILE = "product_data/categories/categories_0001_abcat0010000_to_pcmcat99300050000.xml"

Now we read in our categories data file.

In [8]:
cats = pd.read_xml(CAT_FILE)
cats.head()

We need to find a way to read the nested data so that we are able to interpret the actual path data. We can do this using an element tree to store the different levels of nested data. In the next cell, we will create an element tree and iterate through it to create the path for each category.

In [50]:
prstree = ETree.parse(CAT_FILE)
root = prstree.getroot()
all_cats = []

# Iterate through every path at every level inside of our tree.
for category in root.iter("category"):
    # Only execute the following code if we are looking at a category tag that has a "path"
    # All elements have a name and id so if it has a path it will have length greater than 2.
    if len(category) > 2:
        # Gather the desired information for each category
        cat_id = category.find("id").text

        # Initialize a list of length 5 and fill it with None type.
        levels = [None for i in range(5)]

        # Iterate through everything in the path tag in the category and add
        # each element to the levels list at the appropriate spot.
        i = 0
        for path in category[2]:
            levels[i] = path.find("id").text
            if i < 4:
                i += 1

        # Add the desired data to the proper lists.
        cat_data = [cat_id] + levels
        all_cats.append(cat_data)

# Create a dataframe from the data we acquired in the above loop
xmlToDf = pd.DataFrame(all_cats, columns=['name', 'category', 'category_l1', 'category_l2', 'category_l3', 'category_l4', 'category_l5'])
xmlToDf.head()

,name,category,category_l1,category_l2,category_l3,category_l4,category_l5
0,Gift Center,abcat0010000,cat00000,abcat0010000,None,None,None
1,Her,abcat0011000,cat00000,abcat0010000,abcat0011000,None,None
2,Leisure Gifts,abcat0011001,cat00000,abcat0010000,abcat0011000,abcat0011001,None
3,Kitchen Essentials,abcat0011002,cat00000,abcat0010000,abcat0011000,abcat0011002,None
4,Electronics,abcat0011003,cat00000,abcat0010000,abcat0011000,abcat0011003,None


Now we can read in our test and train datasets.

In [53]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

We can merge our category path data into the train and test datasets.

In [54]:
updated_train = pd.merge(train, xmlToDf, on='category')
updated_test = pd.merge(test, xmlToDf, on='category')
updated_test.head()
updated_train.head()

,user,sku,category,query,click_time,query_time,name,category_l1,category_l2,category_l3,category_l4,category_l5
0,000000df17cd56a5df4a94074e133c9d4739fae3,2125233,abcat0101001,Televisiones Panasonic 50 pulgadas,2011-09-01 23:44:52.533,2011-09-01 23:43:59.752,All Flat-Panel TVs,cat00000,abcat0100000,abcat0101000,abcat0101001,None
1,000001928162247ffaf63185cd8b2a244c78e7c6,2009324,abcat0101001,Sharp,2011-09-05 12:25:37.42,2011-09-05 12:25:01.187,All Flat-Panel TVs,cat00000,abcat0100000,abcat0101000,abcat0101001,None
2,000017f79c2b5da56721f22f9fdd726b13daf8e8,2877125,abcat0101001,rca,2011-10-25 07:18:14.722,2011-10-25 07:16:51.759,All Flat-Panel TVs,cat00000,abcat0100000,abcat0101000,abcat0101001,None
3,0000c4e9d7075985d1020c456e7ce36f83f834eb,2126065,abcat0101001,Samsung 40,2011-09-28 17:27:11.184,2011-09-28 17:26:15.832,All Flat-Panel TVs,cat00000,abcat0100000,abcat0101000,abcat0101001,None
4,00017f7beeac02736c0ce7bf1e75f3010939b34e,1831054,abcat0101001,lcd tv,2011-09-28 07:26:32.153,2011-09-28 07:26:21.476,All Flat-Panel TVs,cat00000,abcat0100000,abcat0101000,abcat0101001,None


Finally, we can write our merged data to new csv files (or we can overwrite the given test.csv and train.csv files)

In [55]:
updated_train.to_csv('new_train.csv')
updated_test.to_csv('new_test.csv')